In [328]:
from Bio.Seq import Seq
from Bio import SeqIO
import numpy as np

In [329]:
data = Bio.SeqIO.parse('s_6.first10000.fastq', 'fastq')
reads = [read.seq for read in data]

In [330]:
class node:
    def __init__(self, k_mer, coverage=None, id_node=None):
        self.k_mer = k_mer
        self.coverage = coverage
        self.edges_in = {}
        self.edges_out = {}
        self.id = id_node
        self.times_delete = 0

class edge:
    def __init__(self, k_mer, coverage=None, node_in=None, node_out=None):
        self.k_mer = k_mer
        self.coverage = coverage
        self.node_in = node_in
        self.node_out = node_out

class graph:
    def __init__(self, reads, k=3):
        self.array_reads = reads
        self.node_kmers = self.k_mers(reads, k)
        self.edges_kmers = self.k_mers(reads, k + 1)
        self.nodes = {} # dictionary k_mer -> class node
        self.k = k
    
    def create(self):
        for index, kmer in enumerate(self.node_kmers):
            self.add_node(kmer, self.node_kmers[kmer], index)
            
        for kmer in self.edges_kmers:
            self.add_edge(kmer, self.edges_kmers[kmer])
    
    def add_node(self, k_mer, coverage, id_node):
        #reverse_complement_k_mer = k_mer.reverse_complement()
        self.nodes[k_mer] = node(k_mer, coverage, id_node)
        #self.nodes[reverse_complement_k_mer] = node(reverse_complement_k_mer, coverage)
        
    def add_edge(self, k_mer, coverage):
        node_1, node_2 = k_mer[:self.k], k_mer[-self.k:] # node_1 -> node_2
        self.nodes[node_1].edges_out[k_mer] = edge(k_mer, coverage, self.nodes[node_1], self.nodes[node_2])
        self.nodes[node_2].edges_in[k_mer] = edge(k_mer, coverage, self.nodes[node_1], self.nodes[node_2])
        
    def delete_node(self, k_mer):
        key_1, key_2 = list(brujne.nodes[k_mer].edges_in.keys())[0], list(brujne.nodes[k_mer].edges_out.keys())[0]
        # Step 1
        A, C, E = brujne.nodes[k_mer].edges_in[key_1].node_in, brujne.nodes[k_mer], brujne.nodes[k_mer].edges_out[key_2].node_out
        # step 2
#         print(A.k_mer, C.k_mer, E.k_mer)
        new_key = key_1 + key_2[self.k:] #new k_mer
        new_cov = (len(A.edges_out[key_1].k_mer) - self.k) * A.edges_out[key_1].coverage
        new_cov += (len(E.edges_in[key_2].k_mer) - self.k) * E.edges_in[key_2].coverage #new coverage
        new_cov = new_cov / (len(A.edges_out[key_1].k_mer) + len(E.edges_in[key_2].k_mer) - self.k * 2)
        A.edges_out.pop(key_1) #delete edge B
        C.edges_in.pop(key_1)
        E.edges_in.pop(key_2) #delete edge D
        C.edges_out.pop(key_2)
        C = None
        #step 3 add new edge
#         print(key_1, key_2)
#         print(new_key)
        self.add_edge(new_key, new_cov)
    
    def k_mers(self, reads, k):
        k_mers_dict = {}
        for read in reads:
            for index in range(len(read) - k + 1):
                sequance_kmer = read[index:index + k]
                reverse_sequance_kmer = sequance_kmer.reverse_complement()
                
                if sequance_kmer in k_mers_dict:
                    k_mers_dict[sequance_kmer] += 1
                else:
                    k_mers_dict[sequance_kmer] = 1
                
                if reverse_sequance_kmer in k_mers_dict:
                    k_mers_dict[reverse_sequance_kmer] += 1
                else:
                    k_mers_dict[reverse_sequance_kmer] = 1
        return k_mers_dict
    
    def compression(self):
        for nodex in brujne.nodes:
            if len(brujne.nodes[nodex].edges_out) == 1 and len(brujne.nodes[nodex].edges_in) == 1:
                brujne.delete_node(nodex)
                
    def print_graph(self):
        for nodex in brujne.nodes:
            for edgex in brujne.nodes[nodex].edges_out:
                cov, len_kmer = brujne.nodes[nodex].edges_out[edgex].coverage, len(brujne.nodes[nodex].edges_out[edgex].k_mer)
                print(str(brujne.nodes[nodex].id), '->' , str(brujne.nodes[nodex].edges_out[edgex].node_out.id) + 
                      f'[label="cov={int(cov)}, len={len_kmer-self.k}"]'';')
    
    def correction(self):
        coverages = []
        for nodex in brujne.nodes:
            for edgex in brujne.nodes[nodex].edges_in:
                coverages.append(brujne.nodes[nodex].edges_in[edgex].coverage)
            for edgex in brujne.nodes[nodex].edges_out:
                coverages.append(brujne.nodes[nodex].edges_out[edgex].coverage)
        COVERAGE_BOUND = np.percentile(coverages, 25)
        LENGHT_BOUND = 2 * self.k

        for nodex in brujne.nodes:
            for edgex in brujne.nodes[nodex].edges_in:
                if len(brujne.nodes[nodex].edges_out) == 0 and len(brujne.nodes[nodex].edges_in) == 1:
                    if len(brujne.nodes[nodex].edges_in[edgex].k_mer) < LENGHT_BOUND or brujne.nodes[nodex].edges_in[edgex].coverage < COVERAGE_BOUND:
                        A = brujne.nodes[nodex].edges_in[edgex].node_in
                        A.edges_out.pop(edgex)
                        brujne.nodes[nodex].edges_in = {}

            for edgex in brujne.nodes[nodex].edges_out:
                if len(brujne.nodes[nodex].edges_out) == 1 and len(brujne.nodes[nodex].edges_in) == 0:
                    if len(brujne.nodes[nodex].edges_out[edgex].k_mer) < LENGHT_BOUND or brujne.nodes[nodex].edges_out[edgex].coverage < COVERAGE_BOUND:
                        C = brujne.nodes[nodex].edges_out[edgex].node_out
                        C.edges_in.pop(edgex)
                        brujne.nodes[nodex].edges_out = {}
    def write_edges(self, file_name):
        with open(file_name, "w") as output_handle:
            for index, nodex in enumerate(brujne.nodes):
                for index_2, edgex in enumerate(brujne.nodes[nodex].edges_out):
                    output_handle.writelines('>' + str(index) + '_' +  str(index_2) + '\n' + edgex + '\n')
    
    def write_graph(self, file_name):
        with open(file_name, "w") as output_handle:
            output_handle.writelines('graph {')
            for nodex in brujne.nodes:
                for edgex in brujne.nodes[nodex].edges_out:
                    cov, len_kmer = brujne.nodes[nodex].edges_out[edgex].coverage, len(brujne.nodes[nodex].edges_out[edgex].k_mer)
                    tmp_edge = str(brujne.nodes[nodex].id), ' -> ' , str(brujne.nodes[nodex].edges_out[edgex].node_out.id) + \
                          f'[label="cov={int(cov)}, len={len_kmer-self.k}"]'';\n'
                    output_handle.writelines(tmp_edge)
            output_handle.writelines('}')
#     def _find_neighbor(self, k_mer):
#         patter_search_in, patter_search_out = k_mer[1:], k_mer[:-1]
#         neigbors_in, neigbors_out = [], []
        
#         for nucleotide in ['A', 'C', 'T', 'G']:
#             if (nucleotide + patter_search_in) in self.nodes:
#                 neigbors_in.append(nucleotide + patter_search_in)
#             if (patter_search_out + nucleotide) in self.nodes:
#                 neigbors_out.append(patter_search_out + nucleotide)
#         return (neigbors_in, neigbors_out)

In [331]:
%%time
brujne = graph(reads, k=55)
brujne.create()
brujne.compression()
brujne.write_edges('edges_without_correction_10000.fasta')
brujne.write_graph('graph_without_correction_10000.dot')
brujne.correction()
brujne.compression()
brujne.write_edges('edges_correction_10000.fasta')
brujne.write_graph('graph_correction_10000.dot')
#brujne.print_graph()

Wall time: 6min 55s


In [327]:
brujne.print_graph()

1758 -> 1866[label="cov=155, len=838"];
1867 -> 1759[label="cov=155, len=838"];
1868 -> 1882[label="cov=28, len=82"];
1883 -> 1869[label="cov=28, len=82"];


c:\users\notebook-kharlamov\appdata\local\programs\python\python37-32\lib\site-packages\Bio\Seq.py:182: BiopythonWarning: Biopython Seq objects now use string comparison. Older versions of Biopython used object comparison. During this transition, please use hash(id(my_seq)) or my_dict[id(my_seq)] if you want the old behaviour, or use hash(str(my_seq)) or my_dict[str(my_seq)] for the new string hashing behaviour.
  BiopythonWarning,
